In [62]:
import music21
import numpy as np

In the code that follows, we define a function that takes input a mid filename as input. We then do the following steps:

1-We add any notes that have the same offset (i.e. they start at the same time), as chords.
2-We give the chord the duration of its shortest non-zero duration.
3-We save all the durations of all notes/chords and return them.
4-We save all the offsets of all notes/chords and return them.
5-We return a list of all unique notes/chords.

notes:

In [100]:
def get_features(filename):
    #offsets of each event
    offsets=[]
    
    #duration of each event
    durations=[]
    
    #note or chord at each event.
    notes=[]
    
    
    notesRaw=music21.converter.parse(filename)
    notesRaw=notesRaw.flat.notes
    for note in notesRaw:  
        if isinstance(note,music21.note.Note):
            notes.append(str(note.pitch.midi))
            durations.append(note.duration.quarterLength)
            offsets.append(note.offset)
        elif isinstance(note,music21.chord.Chord):
            notes.append('.'.join(str(n) for n in note.normalOrder))
            durations.append(note.duration.quarterLength)
            offsets.append(note.offset)
    
    #we now make all notes that are played at the same time as a chord.
    
    
    
    
    new_notes=[]
    new_durations=[]
    uniqueOffsets=list(set(offsets))
    
    #to simplify matters
    notes=np.array(notes)
    durations=np.array(durations)
    offsets=np.array(offsets)

    for eachOffset in uniqueOffsets:
        #take care of creating Chords
        
        notesPlayed=notes[offsets==eachOffset]
        joinedNotes=[]
        for eachNote in notesPlayed:
            joinedNotes.extend(eachNote.split('.'))
        joinedNotes=sorted(set(joinedNotes))
        joined=['.'.join(n for n in joinedNotes)]
        new_notes.extend(joined)
        
        #take care of chosing the right duration:
        new_durations.append(np.max(durations[offsets==eachOffset]))
        
        
        
    
    
    
    return list(new_notes),list(uniqueOffsets),list(new_durations)

In the following function, we create a midi file given a notes list, an offset list, and a durations list:


In [96]:
def write_midi(notes,offsets,durations,filepath):
    entireSong=[]

    for i,note in enumerate(notes):
        if ('.' in note):
            chord=note.split('.')
            individuals=[]
            for n in chord:
                new_note=music21.note.Note(int(n))
                new_note.storedInstrument=music21.instrument.Piano()
                individuals.append(new_note)
            new_chord=music21.chord.Chord(individuals)
            new_chord.offset=offsets[i]
            new_chord.duration=music21.duration.Duration(durations[i])
            entireSong.append(new_chord)
        else:
            new_note=music21.note.Note(int(note))
            new_note.offset=offsets[i]
            new_note.duration=music21.duration.Duration(durations[i])
            new_note.storedInstrument=music21.instrument.Piano()
            entireSong.append(new_note)    
            
    midi_stream=music21.stream.Stream(entireSong)
    midi_stream.write('midi', fp=filepath)
    return 1
    

In [99]:
testNotes,testOffsets,testDurs=get_features('Classical/hay_40_1.mid')
worked=write_midi(testNotes,testOffsets,testDurs,'checkin.mid')


with the above we have changed all notes/chords to numbers.